In [17]:
from langchain_groq import ChatGroq

In [18]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The second person to land on moon was ...")
print(response.content)

The second person to land on the moon was Edwin "Buzz" Aldrin. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, shortly after Neil Armstrong, who was the first person to set foot on the moon.


In [1]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://remoteok.com/remote-jobs/remote-software-engineer-sticker-mule-1093738")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


 



Remote Software Engineer (ğŸ’°~$160k) at Sticker Mule

































 

































			ğŸ‘©â€�ğŸ’» Join Remote OK
		

			ğŸ‘‹  Log in
		


				General
			


 Frontpage
	

ğŸ�� Remote jobs
	

			ğŸŒ—  Dark mode 
		

ğŸ‘©â€�ğŸ’» Hire remote workers
	

ğŸš¨ Post a job	

â­�ï¸� Go premium
		



			Top jobs
		


ğŸ¦¾  AI Jobs


â�°  Async jobs
	

ğŸŒ�  Distributed team
	

					ğŸ¤“ Engineer jobs
				

					ğŸ¤“ Developer jobs
				

					ğŸ�§ Support jobs
				

					ğŸ›  Technical jobs
				

					ğŸ‘µ Senior jobs
				

					ğŸ¤“ Software jobs
				

					ğŸš¥ Marketing jobs
				

					ğŸ’¼ Management jobs
				


			Companies
		


ğŸš¨ Post a remote job
	

ğŸ“¦ Buy a job bundle
	

ğŸ�· Ask for a discount
	
 Health insurance for teams
 Health insurance for nomads


			Feeds
		

ğŸ›  Remote Jobs API
ğŸªš  RSS feed
ğŸª“  JSON feed
  Hacker News mode
  Safe for work mode


			Help
		


		ğŸ’¡  Ideas + bugs
	

		ğŸš€  Changelog
	

		ğŸ›�ï¸�  Merch
	

		ğŸ›

In [9]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

# Load your API key
load_dotenv(dotenv_path="app/.env")

# Define the LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile"
)

# Build the prompt
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)

# Use pipeline
chain_extract = prompt_extract | llm

# Example test (replace page_data with real scraped content)
res = chain_extract.invoke(input={'page_data': "We are hiring Software Engineer with Python, 2 years experience"})
print(res.content)


```json
{
    "role": "Software Engineer",
    "experience": "2 years",
    "skills": ["Python"],
    "description": "We are hiring Software Engineer with Python, 2 years experience"
}
```


In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
job = json_res

In [11]:
job = json_res


In [12]:
type(json_res)

dict

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

In [16]:
import os
import uuid
import pandas as pd

# 🚨 Must be set BEFORE importing chromadb
os.environ["CHROMA_TELEMETRY_ENABLED"] = "false"

import chromadb

# Load your CSV into df
df = pd.read_csv("app/resource/my_portfolio.csv")

# Initialize client & collection
client = chromadb.PersistentClient(path="vectorstore")
collection = client.get_or_create_collection(name="portfolio")

# Safer empty check
try:
    results = collection.get(limit=1)
    is_empty = len(results["ids"]) == 0
except Exception as e:
    print("Collection check failed, assuming empty:", e)
    is_empty = True

if is_empty:
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],       # must be a list
            metadatas={"links": row["Links"]},  # metadata dict
            ids=[str(uuid.uuid4())]             # unique ID
        )


In [17]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [18]:
job

{'role': 'Software Engineer',
 'experience': '2 years',
 'skills': ['Python'],
 'description': 'We are hiring Software Engineer with Python, 2 years experience'}

In [19]:
job = json_res
job['skills']

['Python']

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineer for Your Python Requirements

Dear Hiring Manager,

I came across your job posting for a Software Engineer with 2 years of experience in Python, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With our expertise in Python, we can provide tailored solutions to empower your business, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

At AtliQ, we have a proven track record of delivering high-quality software solutions that meet the unique needs of our clients. Our team of expert software engineers has extensive experience in developing scalable and efficient software applications using Python. I'd like to highlight a few examples of our work:

* https://example.com/ml-python-portfolio: This portfolio showcases our expertise in machine learning and Python, demonstrating our ability to develop complex models and algorithms that drive business value.
* 